In [17]:
import dash 
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
from dash.exceptions import PreventUpdate
import matplotlib.pyplot as plt
file_path ='https://raw.githubusercontent.com/aneesh-barige-ganapathy/vizproject/refs/heads/main/viz%20data/cleaned_file_path.csv'
data = pd.read_csv(file_path)

data = data[data["country"] != "World"]

data_for_matplotlib = pd.read_csv("https://raw.githubusercontent.com/aneesh-barige-ganapathy/vizproject/refs/heads/main/viz%20data/futur_file_path.csv")
years = ['2010', '2020', '2030', '2040', '2050', '2060', '2070']
policy_data = data_for_matplotlib.groupby('Scenario')[years].sum().T

# Calculate top 10 countries based on total emissions
top_10_countries = (
    data.melt(id_vars=["iso", "country", "sector", "gas"], var_name="Year", value_name="Emissions")
    .groupby("country")["Emissions"].sum()
    .nlargest(10)
    .index.tolist()
)

# Melt the dataset to transform year columns into rows
data_melted = data.melt(
    id_vars=["iso", "country", "sector", "gas"],
    var_name="Year",
    value_name="Emissions"
)
data_melted["Year"] = data_melted["Year"].astype(int)  # Convert Year to integer

# Initialize the app
app = dash.Dash(__name__)
server = app.server  # Required for deployment


import dash
from dash import dcc, html

# Assuming 'data', 'data_melted', etc. are already defined or loaded



app.layout = html.Div([
    # Page title
    html.H1(
        "Greenhouse Gas Emissions Dashboard",
        style={
            'textAlign': 'center',
            'color': 'white',
            'marginBottom': '20px'
        }
    ),
    
    # ---------- Pie Charts Row ----------
    html.Div([
        dcc.Loading(
            dcc.Graph(
                id="world-pie-chart",
                style={"width": "600px", "height": "500px", }  # Reduced size
            ),
            type="circle"
        ),
        dcc.Loading(
            dcc.Graph(
                id="pie-chart",
                style={"width": "600px", "height": "500px"}  # Reduced size
            ),
            type="circle"
        ),
    ], style={
        'display': 'flex',
        'justifyContent': 'space-evenly',
        'marginBottom': '30px'
    }),
    
    # ---------- Bar Chart Section ----------
    html.Div([
        html.H2(
            "Bar Chart",
            style={'color': 'white', 'marginBottom': '10px'}
        ),
        
        html.Div([
            html.Label("Select Countries:"),
            dcc.Dropdown(
                id="bar-chart-country-dropdown",
                options=[{'label': c, 'value': c} for c in data["country"].unique()],
                multi=True,
                value=[]
            ),

            html.Label("Select Year:"),
            dcc.Dropdown(
                id="bar-chart-year-dropdown",
                options=[{'label': y, 'value': y} for y in sorted(data_melted["Year"].unique())],
                value=data_melted["Year"].max()
            ),

            html.Label("Select Gas:"),
            dcc.Dropdown(
                id="bar-chart-gas-dropdown",
                options=[{'label': g, 'value': g} for g in data["gas"].unique()],
                value='All GHG'
            ),
        ], style={
            'width': '20%',
            'display': 'inline-block',
            'verticalAlign': 'top',
            'paddingRight': '20px'
        }),

        html.Div([
            dcc.Loading(
                dcc.Graph(id="bar-chart"),
                type="circle"
            ),
        ], style={'width': '75%', 'display': 'inline-block'}),
    ], style={'marginBottom': '40px'}),
    
    # ---------- Line Chart Section ----------
    html.Div([
        html.H2(
            "Line Chart",
            style={'color': 'white', 'marginBottom': '10px'}
        ),

        html.Div([
            html.Label("Select Countries:"),
            dcc.Dropdown(
                id="line-chart-country-dropdown",
                options=([{'label': "Top 10 Countries", 'value': 'top_10'}] +
                         [{'label': c, 'value': c} for c in data["country"].unique()]),
                multi=True,
                value=['top_10']
            ),

            html.Label("Select Sector:"),
            dcc.Dropdown(
                id="sector-dropdown",
                options=[{'label': s, 'value': s} for s in data["sector"].unique()],
                value='Total including LUCF'
            ),

            html.Label("Select Gas Type:"),
            dcc.Dropdown(
                id="gas-dropdown",
                options=[{'label': g, 'value': g} for g in data["gas"].unique()],
                value='All GHG'
            ),
        ], style={
            'width': '20%',
            'display': 'inline-block',
            'verticalAlign': 'top',
            'paddingRight': '20px'
        }),

        html.Div([
            dcc.Loading(
                dcc.Graph(id="line-chart"),
                type="circle"
            ),
        ], style={'width': '75%', 'display': 'inline-block'}),
    ], style={'marginBottom': '40px'}),

    # ---------- Choropleth Map Section ----------
    html.Div([
        html.H2(
            "Choropleth Map",
            style={'color': 'white', 'marginBottom': '10px'}
        ),
        
        html.Div([
            html.Label("Select Gas Type:"),
            dcc.Dropdown(
                id="choropleth-gas-dropdown",
                options=[{'label': g, 'value': g} for g in data["gas"].unique()],
                value='All GHG'
            ),

            html.Label("Select Year:"),
            dcc.Slider(
                id="choropleth-year-slider",
                min=data_melted["Year"].min(),
                max=data_melted["Year"].max(),
                step=1,
                marks={
                    y: str(y) for y in range(data_melted["Year"].min(),
                                             data_melted["Year"].max() + 1, 5)
                },
                value=data_melted["Year"].max()
            ),
        ], style={
            'width': '20%',
            'display': 'inline-block',
            'verticalAlign': 'top',
            'paddingRight': '20px'
        }),

        html.Div([
            dcc.Loading(
                dcc.Graph(id="choropleth-map"),
                type="circle"
            ),
        ], style={'width': '75%', 'display': 'inline-block'}),
    ], style={'marginBottom': '40px'}),
    
    # ---------- Future Graph Section ----------
    html.Div([
        html.H2(
            "Future Predictions",
            style={'color': 'white', 'marginBottom': '10px'}
        ),
        dcc.Loading(
            dcc.Graph(id="future-graph"),
            type="circle"
        ),
    ]),

], style={
    'color': 'white',
    'backgroundColor': '#2C2C2C',
    'padding': '20px'
})
     
          
# Callbacks
@app.callback(
    [Output("line-chart", "figure"),
     Output("choropleth-map", "figure")],
    [Input("line-chart-country-dropdown", "value"),
     Input("sector-dropdown", "value"),
     Input("gas-dropdown", "value"),
     Input("choropleth-gas-dropdown", "value"),
     Input("choropleth-year-slider", "value")]
)
def update_dashboard(selected_countries, selected_sector, selected_gas, choropleth_gas, choropleth_year):
    # Handle Top 10 countries selection
    if 'top_10' in selected_countries or not selected_countries:
        selected_countries = top_10_countries

    # Filter the data for line chart
    line_chart_data = data_melted[
        (data_melted["sector"] == selected_sector) &
        (data_melted["gas"] == selected_gas) &
        (data_melted["country"].isin(selected_countries))
    ]

    # Handle empty data for line chart
    if line_chart_data.empty:
        line_chart = px.line(title="No Data Available")
    else:
        line_chart = px.line(
            line_chart_data, x="Year", y="Emissions", color="country",
            title=f"Emissions Over Time: {selected_sector} - {selected_gas}",
            markers=True
        )

    # Filter data for choropleth map
    choropleth_data = data_melted[
        (data_melted["gas"] == choropleth_gas) &
        (data_melted["Year"] == choropleth_year) &
        (data_melted["sector"] == 'Total including LUCF')
    ]

    # Handle empty data for choropleth map
    if choropleth_data.empty:
        choropleth_map = px.choropleth(title="No Data Available")
    else:
        choropleth_map = px.choropleth(
            choropleth_data,
            locations="iso",
            color="Emissions",
            hover_name="country",
            title=f"Total Emissions from all sectors Globally ({choropleth_year}, {choropleth_gas})",
            range_color=[100, 8000],
            color_continuous_scale="reds"
        )

    return line_chart, choropleth_map

@app.callback(
    Output("bar-chart", "figure"),
    [Input("bar-chart-country-dropdown", "value"),
     Input("bar-chart-year-dropdown", "value"),
     Input("bar-chart-gas-dropdown", "value")]
)
def update_bar_chart(selected_countries, selected_year, selected_gas):
    if 'top_10' in selected_countries or not selected_countries:
        selected_countries = top_10_countries
    
    # Filter data for the bar chart
    bar_chart_data = data_melted[
        (data_melted["Year"] == selected_year) & 
        (data_melted["gas"] == selected_gas) &
        (data_melted["sector"] != "Total including LUCF")
    ]
    if selected_countries:
        bar_chart_data = bar_chart_data[bar_chart_data["country"].isin(selected_countries)]

    # Handle empty data for bar chart
    if bar_chart_data.empty:
        return px.bar(title="No Data Available")

    # Create stacked bar chart
    bar_chart = px.bar(
        bar_chart_data, x="country", y="Emissions", color="country",
        title=f"Total Emissions by Country and Sector ({selected_year}, {selected_gas})",
        labels={"Emissions": "Total Emissions"},
        barmode="stack",
        hover_data=["sector"]
    )

    return bar_chart

@app.callback(
    [Output("pie-chart", "figure"),
     Output("world-pie-chart", "figure")],
    [Input("choropleth-year-slider", "value")]
)
def update_pie_charts(selected_year):
    latest_year = data_melted["Year"].max()

    # Filter data for the sector-wise pie chart
    sector_pie_data = data_melted[
        (data_melted["Year"] == selected_year) &
        (data_melted["sector"] != "Total including LUCF")
    ].groupby("sector")["Emissions"].sum().reset_index()

    # Handle empty data for sector pie chart
    if sector_pie_data.empty:
        sector_pie_chart = px.pie(title="No Data Available")
    else:
        sector_pie_chart = px.pie(
            sector_pie_data, 
            names="sector", 
            values="Emissions", 
            title=f"Sector-wise Emissions ({latest_year})"
        )
    file_path = 'https://raw.githubusercontent.com/aneesh-barige-ganapathy/vizproject/refs/heads/main/viz%20data/cleaned_file_path.csv'
    data = pd.read_csv(file_path)
    # Filter data for the world greenhouse gas composition pie chart
    world_pie_data = data[data["country"] == "World"].melt(
        id_vars=["iso", "country", "sector", "gas"],
        var_name="Year",
        value_name="Emissions"
    )
    world_pie_data = world_pie_data[
        (world_pie_data["Year"] == str(selected_year)) &
        (world_pie_data["sector"] == "Total including LUCF")&
        (world_pie_data["gas"]!="All GHG")
    ].groupby("gas")["Emissions"].sum().reset_index()

    # Handle empty data for world pie chart
    if world_pie_data.empty:
        world_pie_chart = px.pie(title="No Data Available")
    else:
        world_pie_chart = px.pie(
            world_pie_data, 
            names="gas", 
            values="Emissions", 
            title=f"Global Greenhouse Gas Composition ({latest_year})"
        )

    return sector_pie_chart, world_pie_chart
 #Plotting
@app.callback(
    Output("future-graph", "figure"),
    Input("choropleth-year-slider", "value")
)
def update_future_graph(selected_year):
    fig = px.line(
        policy_data.reset_index(),
        x='index',
        y=policy_data.columns,
        title="Effect of Policy on CO2 Emissions Over the Years",
        labels={"index": "Year", "value": "Total Emissions(CO2)"},
        markers=True
    )
    return fig
# Run the app
if __name__ == "__main__":
    app.run_server(debug=True,port=112)
